<a href="https://colab.research.google.com/github/marichaf/crf-incentives/blob/main/CRF_Incentive_Reporting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CRF Icentive recipient reporting
# Created 2/19/25 by Maricha
  # Updated 3/6/25

In [ ]:
# This block is required for use on Google Drive only. DO NOT EDIT

from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from tabulate import tabulate
import csv

## Edit the cell below where indicated:

In [ ]:
### THIS IS THE ONLY CELL THAT REQUIRES EDITING TO RUN THE SCRPT! ###

# Read the CSV file into a pandas dataframe
shared_drive_path = '/content/drive/Shared drives/Service Delivery/ETO Reports/Python Scripts/CRF Incentives/' # Only update if the filepath has changed

# INSERT FILE NAME(S) HERE: Keep apostrophes and .csv. Make sure it matches exactly with the file name in the folder ^.
# Import State Funded Services Report
state_funded_services = 'SFS 1-23 to 4-8-2025.csv'

# Program enrollments report
enrollments_report = 'PER 1-23 to 4-8-2025.csv'



In [ ]:
today_date = pd.Timestamp('today').normalize().date()
print(today_date)

2025-04-09


In [14]:
# checking for the need to skip columns


def read_csv_dynamic_header(filepath, expected_columns, encoding_options=None):
    """
    Reads a CSV file by dynamically identifying the header row containing the expected columns,
    trying multiple encodings if necessary.

    Parameters:
        filepath (str): Path to the CSV file.
        expected_columns (list): A list of expected column names.
        encoding_options (list, optional): A list of encoding strings to try.
            Defaults to ['utf-8', 'cp1252', 'iso-8859-1', 'utf-8-sig'].

    Returns:
        DataFrame: A pandas DataFrame read with the correct header row and encoding.

    Raises:
        ValueError: If none of the encodings result in detecting the expected header row.
    """

    # Default list of encodings to try if none are provided.
    if encoding_options is None:
        encoding_options = ['utf-8', 'cp1252', 'iso-8859-1', 'utf-8-sig']

    # Normalize the expected columns for robust matching.
    expected = set(col.strip().lower() for col in expected_columns)

    # Try each encoding option.
    for enc in encoding_options:
        header_idx = None

        try:
            # Open the file with the current encoding. Use errors='replace' to avoid crashes.
            with open(filepath, 'r', encoding=enc, errors='replace') as f:
                reader = csv.reader(f)
                for i, row in enumerate(reader):
                    # Normalize current row cell values.
                    normalized_row = set(cell.strip().lower() for cell in row if cell.strip())
                    if expected.issubset(normalized_row):
                        header_idx = i
                        break
        except Exception as e:
            print(f"Error trying encoding '{enc}': {e}")
            continue  # Try the next encoding option if reading fails.

        if header_idx is not None:
            try:
                # Read the CSV with pandas using the determined header row and encoding.
                df = pd.read_csv(filepath, header=header_idx, encoding=enc, encoding_errors='replace')
                print(f"Successfully read the file using '{enc}' encoding with header at row {header_idx}.")
                return df
            except Exception as e:
                print(f"Failed to read CSV using encoding '{enc}' after header detection: {e}")
                continue

    # If no encoding produced a valid header detection, raise an error.
    raise ValueError("Could not find the expected header row using any of the provided encodings. "
                     "Please verify the file format and ensure that it contains the following columns: "
                     + ", ".join(expected_columns))

    # Now read the CSV using pandas by specifying the header row
    df = pd.read_csv(filepath, header=header_idx, encoding=file_encoding, encoding_errors='replace')
    return df

# Example usage:
expected_cols = ['WDA', 'Office', 'Case Number', 'Name', 'Program of Enrollment',
       'Is the participant Above 200% FPL', 'Service Provided',
       'Type of Resource', 'Other Resource', 'Date TP Entered',
       'Last Updated By_346', 'Training Program', 'Training Provider',
       'Start Date', 'End Date', 'Actual Outcome', 'Amount',
       'Occupation Title']  # headings from state-funded services report


def read_csv_with_multiple_encodings(filepath):
    encoding_options = ['utf-8', 'cp1252', 'utf-8-sig']  # utf-8-sig handles files with BOM if applicable.
    for enc in encoding_options:
        try:
            df = pd.read_csv(filepath, encoding=enc, low_memory=False)
            print(f"Successfully read the file using {enc} encoding.")
            return df
        except UnicodeDecodeError:
            print(f"Decoding using {enc} failed. Trying the next encoding...")
        except Exception as e:
            print(f"An error occurred with encoding {enc}: {e}")
    print("Failed to read the CSV file with the expected encodings. Please verify the file format.")
    return None



In [15]:
file_path_state_funded = f'{shared_drive_path}{state_funded_services}'
#data = pd.read_csv(file_path_state_funded, low_memory=False, encoding='utf-8', encoding_errors='replace') #skiprows=3 because of the weird formatting from ETO, potentially change to Excel
#print(f'State Funded Services report has {data.shape[0]} rows and {data.shape[1]} columns')

try:
    data = read_csv_dynamic_header(file_path_state_funded, expected_cols)
    print(f'State Funded Services report has {data.shape[0]} rows and {data.shape[1]} columns')
    # Proceed with processing your DataFrame.
except Exception as e:
    print("An error occurred while loading the CSV:", e)

file_path_enrollments = f'{shared_drive_path}{enrollments_report}'
enrollments = read_csv_with_multiple_encodings(file_path_enrollments)
print(f'Enrollments report has {enrollments.shape[0]} rows and {enrollments.shape[1]} columns')

Successfully read the file using 'utf-8' encoding with header at row 0.
State Funded Services report has 2148 rows and 18 columns
Successfully read the file using utf-8 encoding.
Enrollments report has 9954 rows and 79 columns


In [ ]:
data.columns

Index(['WDA', 'Office', 'Case Number', 'Name', 'Program of Enrollment',
       'Is the participant Above 200% FPL', 'Service Provided',
       'Type of Resource', 'Other Resource', 'Date TP Entered',
       'Last Updated By_346', 'Training Program', 'Training Provider',
       'Start Date', 'End Date', 'Actual Outcome', 'Amount',
       'Occupation Title'],
      dtype='object')

In [ ]:
# Convert the 'Date' column to a datetime format across all dfs
data['Date TP Entered'] = pd.to_datetime(data['Date TP Entered'])

# ************
enrollments['Enrollment End Date'] = pd.to_datetime(enrollments['Enrollment End Date'])
enrollments['Enrollment Start Date'] = pd.to_datetime(enrollments['Enrollment Start Date'])

In [ ]:
# Filter for all rows where "Service Provided" == "Community Reinvestment Financial Support Payments"
filtered_data = data.loc[data["Service Provided"] == "Community Reinvestment Financial Support Payments"]
print(f'Incentive payments (before aggregating duplicates): {filtered_data.shape[0]}')

# "remove" Duplicates - really we are just aggregating them so we don't lose any information.
def aggregate_to_list(values):
    return sorted(set(map(str, values)))

fields_to_aggregate = ['Service Provided', 'Date TP Entered', 'Amount']

# Create an aggregation dictionary for the desired fields
agg_logic = {field: aggregate_to_list for field in fields_to_aggregate}

# Group by 'ETO Case Number' and apply the aggregation logic
filtered_data_agg = filtered_data.groupby('Case Number').agg(agg_logic).reset_index()

print(f'Unique ETO numbers: {filtered_data_agg.shape[0]}')

Incentive payments (before aggregating duplicates): 405
Unique ETO numbers: 90


In [ ]:
# need to also aggregate enrollments because duplicates :/
fields_to_aggregate_enrollments = ['Program Name', 'Enrollment Start Date', 'Enrollment End Date', 'Ethnicity', 'Race']

# Create an aggregation dictionary for the desired fields
agg_logic_enrollments = {field: aggregate_to_list for field in fields_to_aggregate_enrollments}

# Group by 'ETO Case Number' and apply the aggregation logic
enrollments_agg = enrollments.groupby('ETO Case Number').agg(agg_logic_enrollments).reset_index()

enrollments_agg.shape

(7712, 6)

In [ ]:
# Rename ETO field for merge:
enrollments_agg['Case Number'] = enrollments_agg['ETO Case Number']

In [ ]:
crf_demographics = filtered_data_agg.merge(
    enrollments_agg[["Case Number", "Program Name", 'Enrollment Start Date', 'Enrollment End Date', 'Ethnicity', 'Race']],
    on="Case Number",
    how="left"
)

In [ ]:
crf_demographics.head()

,Case Number,Service Provided,Date TP Entered,Amount,Program Name,Enrollment Start Date,Enrollment End Date,Ethnicity,Race
0,41080,[Community Reinvestment Financial Support Paym...,"[2024-04-04 00:00:00, 2024-04-12 00:00:00, 202...","[$1,000.00 ]","[QUEST NDWG, State-funded Economic Security fo...","[2023-06-13 00:00:00, 2023-08-09 00:00:00, 202...","[2024-07-02 00:00:00, 2024-07-26 00:00:00]",[Not Hispanic or Latino],[White]
1,103064,[Community Reinvestment Financial Support Paym...,"[2025-01-23 00:00:00, 2025-02-27 00:00:00]","[$1,000.00 ]",[State-funded Economic Security for All (EcSA)],[2024-12-19 00:00:00],[NaT],[Not Hispanic or Latino],[White]
2,116074,[Community Reinvestment Financial Support Paym...,"[2024-05-23 00:00:00, 2024-06-04 00:00:00, 202...","[$1,000.00 ]","[QUEST NDWG, State-funded Economic Security fo...","[2024-02-16 00:00:00, 2024-04-18 00:00:00]","[2025-02-26 00:00:00, NaT]",[Hispanic or Latino],[Decline to Identify]
3,125476,[Community Reinvestment Financial Support Paym...,"[2024-07-25 00:00:00, 2024-08-29 00:00:00, 202...","[$1,000.00 ]","[QUEST NDWG, State-funded Economic Security fo...","[2024-07-10 00:00:00, 2024-07-12 00:00:00]",[NaT],[Not Hispanic or Latino],[White]
4,162910,[Community Reinvestment Financial Support Paym...,"[2024-05-06 00:00:00, 2024-06-10 00:00:00, 202...","[$1,000.00 ]",[State-funded Economic Security for All (EcSA)...,"[2024-04-04 00:00:00, 2024-04-05 00:00:00]",[2024-08-29 00:00:00],[Not Hispanic or Latino],[Black/African American]


In [ ]:
crf_demographics["Case Number"].is_unique

True

In [ ]:
# Ensure your date column is in datetime format

# Explode both the 'Service Provided' and 'Date TP Entered' columns simultaneously.
exploded = crf_demographics.explode(["Date TP Entered"])
exploded['Date TP Entered'] = pd.to_datetime(exploded['Date TP Entered'].astype(str))
# Define your quarter assignment function (adjust boundaries as needed)
def assign_program_quarter(date):
    if pd.Timestamp('2023-07-01') <= date <= pd.Timestamp('2023-09-30'):
        return 'PY23 Q1'
    elif pd.Timestamp('2023-10-01') <= date <= pd.Timestamp('2023-12-31'):
        return 'PY23 Q2'
    elif pd.Timestamp('2024-01-01') <= date <= pd.Timestamp('2024-03-31'):
        return 'PY23 Q3'
    elif pd.Timestamp('2024-04-01') <= date <= pd.Timestamp('2024-06-30'):
        return 'PY23 Q4'
    elif pd.Timestamp('2024-07-01') <= date <= pd.Timestamp('2024-09-30'):
        return 'PY24 Q1'
    elif pd.Timestamp('2024-10-01') <= date <= pd.Timestamp('2024-12-31'):
        return 'PY24 Q2'
    elif pd.Timestamp('2025-01-01') <= date <= pd.Timestamp('2025-03-31'):
        return 'PY24 Q3'
    elif pd.Timestamp('2025-04-01') <= date <= pd.Timestamp('2025-06-30'):
        return 'PY24 Q4'
    else:
        return None  # or flag as 'Outside Fiscal Range'

# Apply the quarter categorization function
exploded['Program Quarter'] = exploded['Date TP Entered'].apply(assign_program_quarter)

# Now you have each service event flagged with the correct quarter.
# You can proceed to group or count unique cases by quarter.
quarter_counts = exploded.groupby('Program Quarter')['Case Number'].nunique().reset_index()
quarter_counts.columns = ['Program Quarter', 'Unique Count']

# If you also want a YTD summary (i.e. overall count)
ytd_count = exploded['Case Number'].nunique()
# Create a new DataFrame for the YTD row and use pd.concat to combine:
ytd_df = pd.DataFrame([{'Program Quarter': 'Program start to date', 'Unique Count': ytd_count}])
quarter_counts = pd.concat([quarter_counts, ytd_df], ignore_index=True)

print("Number of unique participants that received CRF Incentives, Total")
display(quarter_counts)


Number of unique participants that received CRF Incentives, Total


,Program Quarter,Unique Count
0,PY23 Q3,9
1,PY23 Q4,30
2,PY24 Q1,38
3,PY24 Q2,39
4,PY24 Q3,58
5,PY24 Q4,20
6,Program start to date,90


In [ ]:

# Explode the 'Date TP Entered' column
exploded = crf_demographics.explode(["Date TP Entered"])
exploded['Date TP Entered'] = pd.to_datetime(exploded['Date TP Entered'].astype(str))

# Define the quarter assignment function (no need to redefine, same as above)

# Apply quarter assignment
exploded['Program Quarter'] = exploded['Date TP Entered'].apply(assign_program_quarter)

# Define a function to check the priority population criteria
def check_priority_population(row):
    # Get the values from the row
    ethnicity = row['Ethnicity']
    race = row['Race']

    # Check ethnicity:
    if isinstance(ethnicity, list):
        is_hispanic = "Hispanic or Latino" in ethnicity
    else:
        is_hispanic = ethnicity == "Hispanic or Latino"

    # Define target race criteria
    target_races = ["Black/African American", "American Indian/Alaska Native"]
    if isinstance(race, list):
        is_priority_race = any(target in race for target in target_races)
    else:
        is_priority_race = any(target in race for target in target_races)

    return is_hispanic or is_priority_race

# Create the Priority Population flag column
exploded['Priority Population'] = exploded.apply(check_priority_population, axis=1)

# Filter the dataset for rows meeting the priority criteria
filtered = exploded[exploded['Priority Population']]

# Group by Program Quarter and count unique Case Numbers
quarter_counts = filtered.groupby('Program Quarter')['Case Number'].nunique().reset_index()
quarter_counts.columns = ['Program Quarter', 'Unique Count']

# Compute the overall unique count (YTD) across all quarters
ytd_count = filtered['Case Number'].nunique()
ytd_df = pd.DataFrame([{'Program Quarter': 'Program start to date', 'Unique Count': ytd_count}])
quarter_counts = pd.concat([quarter_counts, ytd_df], ignore_index=True)

print("Number of unique participants that received CRF Incentives AND represent the priority populations")
display(quarter_counts)


Number of unique participants that received CRF Incentives AND represent the priority populations


,Program Quarter,Unique Count
0,PY23 Q3,2
1,PY23 Q4,8
2,PY24 Q1,12
3,PY24 Q2,10
4,PY24 Q3,33
5,PY24 Q4,13
6,Program start to date,44


# Final Counts:
Please note:
Each quarter's count represents unique individuals for that specific quarter, meaning within Q3, those 34 are all unique. However, many individuals may have received services in more than one quarter. As a result, when you combine them for a year-to-date (YTD) count, overlapping individuals are only counted once, which is why the YTD total is less than the sum of the individual quarterly counts.

In [ ]:
# Exporting the final dataset as csv so you can review it:
export_filename = f"{shared_drive_path}"+f"crf_incentives_unique"+f"_{today_date}.csv"
crf_demographics.to_csv(export_filename, index=False)

# Non-uninque dataset:
export_nonunique = f"{shared_drive_path}"+f"crf_incentives_non_unique"+f"_{today_date}.csv"
exploded.to_csv(export_nonunique, index=False)